In [195]:
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci
from bs4 import BeautifulSoup

In [18]:
SCORE_DIR =  'data0/scores'

In [30]:
box_scores = os.listdir(SCORE_DIR)

In [36]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith ('.html')]

In [40]:
def parse_html (box_score):
    with open (box_score) as f: 
        html = f.read()
    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select('tr.over_header')]
    [s.decompose() for s in soup.select('tr.thead')]
    return soup

In [60]:
def read_line_score (soup):
    line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = 'team'
    cols[-1] = 'total'
    line_score.columns = cols

    line_score = line_score[['team','total']]
    return line_score

In [110]:
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]
    df = df.apply(pd.to_numeric, errors='coerce')
    return df

In [170]:
def read_season_info (soup):
    nav = soup.select('#bottom_nav_container')[0]
    href = [a['href'] for a in nav.find_all('a')]
    season =  os.path.basename(href[1]).split('_')[0]
    return season

In [187]:
base_cols = None
box_score = box_scores[0]
games = []
for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)
    teams = list(line_score['team'])
    
    summaries = []
    for team in teams: 
        basic = read_stats(soup, team, 'basic')
        advanced = read_stats(soup, team, 'advanced')
    
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower() # making column names lowercase
        
        maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max() ] )
        maxes.index = maxes.index.str.lower() + '_max' # adding 'max' to the column name because the names are the same in total and we want to distinguish
    
        summary = pd.concat([totals, maxes])
    
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep='first'))
            base_cols = [b for b in base_cols if 'bpm' not in b]
    
        summary = summary[base_cols]
    
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T
    
    game = pd.concat([summary, line_score], axis=1)
    
    game['home'] = [0, 1]
    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += '_opp'
    
    full_game = pd.concat([game, game_opp], axis=1)
    
    full_game['season'] = read_season_info(soup)
    
    full_game['date'] = os.path.basename(box_score)[:8]
    full_game['date'] = pd.to_datetime(full_game['date'], format='%Y%m%d')
    
    
    full_game['won'] = full_game['total'] > full_game['total_opp']
    games.append(full_game)

    if len(games) % 40 == 0:
        print(f'{len(games)} / {len(box_scores)}')

/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2867914958.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]

40 / 93


/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2867914958.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]

80 / 93


/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2867914958.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  line_score = pd.read_html(str(soup), attrs={'id': 'line_score'})[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]
/var/folders/dt/_vx_4f6x7rx7_zdn2gc3sn1r0000gp/T/ipykernel_64607/2840185209.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'},index_col=0)[0]

In [191]:
games_df = pd.concat(games, ignore_index=True)

In [199]:
games_df.head(15)

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,42.0,92.0,0.457,17.0,41.0,0.415,23.0,29.0,...,25.0,34.5,160.0,124.0,POR,121,1,2022,2021-10-20,True
1,240.0,240.0,45.0,93.0,0.484,12.0,35.0,0.343,19.0,22.0,...,16.7,36.4,175.0,120.0,SAC,124,0,2022,2021-10-20,False
2,240.0,240.0,39.0,83.0,0.470,14.0,40.0,0.350,14.0,18.0,...,27.3,28.0,138.0,118.0,OKC,98,1,2022,2021-10-26,True
3,240.0,240.0,36.0,86.0,0.419,12.0,34.0,0.353,14.0,17.0,...,66.7,31.3,181.0,113.0,GSW,106,0,2022,2021-10-26,False
4,240.0,240.0,46.0,92.0,0.500,11.0,35.0,0.314,12.0,14.0,...,33.3,28.4,228.0,120.0,OKC,123,1,2022,2021-10-27,False
5,240.0,240.0,43.0,86.0,0.500,15.0,37.0,0.405,22.0,29.0,...,31.9,37.5,228.0,129.0,LAL,115,0,2022,2021-10-27,True
6,240.0,240.0,35.0,86.0,0.407,13.0,39.0,0.333,19.0,27.0,...,66.7,43.1,181.0,111.0,PHI,110,1,2022,2021-10-28,False
7,240.0,240.0,38.0,76.0,0.500,12.0,27.0,0.444,22.0,24.0,...,40.0,37.3,137.0,121.0,DET,102,0,2022,2021-10-28,True
8,240.0,240.0,38.0,96.0,0.396,8.0,29.0,0.276,22.0,26.0,...,23.1,31.1,125.0,116.0,BRK,93,1,2022,2021-10-27,True
9,240.0,240.0,33.0,85.0,0.388,14.0,43.0,0.326,13.0,14.0,...,25.3,46.5,200.0,98.0,MIA,106,0,2022,2021-10-27,False


In [201]:
games_df.to_csv('nba_game_data.csv')